In [ ]:
!pip install boto3 tqdm

In [ ]:
import boto3
import os
from boto3.s3.transfer import TransferConfig
from tqdm import tqdm

In [ ]:
MODEL_REPO = "raft/granite-4.0-micro-raft"
directory = '/opt/app-root/src/raft-workshop/outputs/granite-4.0-micro-raft-full'
key_base = f'models/{MODEL_REPO}'
bucket_name = os.getenv("AWS_S3_BUCKET")

In [ ]:
config = TransferConfig(multipart_threshold=1024 * 25,  # 25MB
                        max_concurrency=10,
                        multipart_chunksize=1024 * 25,  # 25MB
                        use_threads=True)

In [ ]:
for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)

        rel_path = os.path.relpath(file_path, directory)
        key_name = os.path.join(key_base, rel_path)

        print(key_name)
        try:
            with tqdm(
                total=os.path.getsize(file_path),
                unit='B',
                unit_scale=True,
                desc=file_path
            ) as pbar:
                s3_client = boto3.client(
                    's3',
                    endpoint_url="http://minio-api.minio-operator.svc.cluster.local:9000",
                    aws_access_key_id="minioadmin",
                    aws_secret_access_key="ee93o3Gl8A"
                )
                s3_client.upload_file(
                    file_path,
                    "test",
                    key_name,
                    Config=config,
                    Callback=lambda bytes_transferred: pbar.update(bytes_transferred)
                )
            print(f'File {file_path} uploaded to {bucket_name}/{key_name}')
        except Exception as e:
            print(f'Error occurred while uploading {file_path}: {e}')
